#**RAG Chatbot 정답**
#### **PDF 파일의 정보를 추출하고 RAG 기법을 사용하여 이 정보를 LLM이 활용하는 챗봇 만들기**

##**1. 환경 준비**

#### (1) 구글 드라이브 연결

In [ ]:
# 구글 드라이브를 코랩 환경에 연결하여 파일을 저장하거나 읽을 수 있도록 설정
from google.colab import drive
drive.mount('/content/drive')

#### (2) 라이브러리 설치

In [ ]:
!pip install streamlit -q
!pip install pyngrok -q
!pip install openai -q
!pip install langchain -q
!pip install -U langchain-community -q
!pip install sentence-transformers -q
!pip install faiss-gpu -q
!pip install pyPDF2 -q

##**2. 코딩**

In [ ]:
# 이 파일에 Streamlit 애플리케이션 코드를 작성
# 코드 수정 후, 셀을 실행해야 변경된 내용이 ooo.py 파일이 적용됨
%%writefile /content/drive/MyDrive/Streamlit/6-3.RAGChatbot.py

# 라이브러리 불러오기
import streamlit as st
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PDF 문서에서 텍스트 추출 함수 정의하기
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# 텍스트 분할 함수 정의하기
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        separators="\\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

# 임베딩 생성 & 벡터 저장소 생성 함수 정의하기
def get_vectorstore(text_chunks):
    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

# openai 키 입력
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxx"

# 주어진 벡터 저장소로 대화 체인 생성 함수 정의하기
def get_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_message=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k'),
        retriever=vectorstore.as_retriever(),
        get_chat_history=lambda h: h,
        memory=memory
    )
    return conversation_chain

# PDF 파일을 업로드하고 텍스트를 추출하여 AI 대화 준비하기
user_uploads = st.file_uploader("파일을 업로드해주세요~", accept_multiple_files=True)
if st.button("Upload"):
    with st.spinner("처리중.."):
        raw_text = get_pdf_text(user_uploads)
        text_chunks = get_text_chunks(raw_text)
        vectorstore = get_vectorstore(text_chunks)
        st.session_state.conversation = get_conversation_chain(vectorstore)

# 사용자의 질문에 AI가 답변하기
if user_query := st.chat_input("질문을 입력해주세요~"):
    if 'conversation' in st.session_state:
        result = st.session_state.conversation({
            "question": user_query,
            "chat_history": st.session_state.get('chat_history', [])
        })
        response = result["answer"]
    else:
        response = "먼저 문서를 업로드해주세요~."
    with st.chat_message("assistant"):
        st.write(response)

##**3. Streamlit 실행**
* **ngrok 인증 토큰 받는 방법**
    * ngrok.com 사이트 가입 (Sign up)
    * 좌측 Getting Started > Your Authtoken 메뉴 선택하여 토큰 번호 받음
    * ngrok은 인터넷으로 터널을 만들어 로컬 개발 환경의 포트를 개방하고 애플리케이션의 외부 접근을 도와주는 터널 프로그램
*   **ooo.py 파일을 수정했는 데 이전 결과가 그대로 나타남**  
    * 코드 수정 후, 셀을 실행해야 변경된 내용이 ooo.py 파일이 적용됨

* **아래 셀 실행결과가 아래와 같이 나타나면 좌측 url 주소(& <U>Visit Site 버튼</U> 클릭) 선택하여 결과 확인**
    * NgrokTunnel: "https://5c10-34-86-195-94.ngrok-free.app" -> "http://localhost:8501"

In [ ]:
from pyngrok import ngrok

# ngrok 인증 토큰을 설정
ngrok.set_auth_token("ngrok 인증토큰번호를 입력하세요")

# Streamlit 애플리케이션을 백그라운드에서 실행
# ngrok을 통해 외부에서 접근 가능한 URL을 생성
!streamlit run /content/drive/MyDrive/Streamlit/6-3.RAGChatbot.py&>/dev/null&
url = ngrok.connect( addr='8501' )
print(url)

###**참조)**

In [ ]:
# 실행 중인 세션 확인
!ps

In [ ]:
# 사용하는 라이브러리와 버전 확인하기
!pip list